In [1]:
from common_imports import *
from functions import (
    create_subset,
    filter_datasets_for_use_case,
    split_datasets,
    unique_elements,
    PerplexityLoggingCallback,
    print_trainable_parameters,
    CustomDataset,
    get_sequences,
    evaluate,
    CustomTrainer
)

/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-02 20:48:11.529505: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115_nocublaslt.so...


/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-I/root')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('2')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-L/root/boost_1_75_0/stage/lib')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/var/lib/snap')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("resources/train.json", "r") as f:
    args = json.load(f)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=args["load_in_4bit"],
    bnb_4bit_use_double_quant=args["bnb_4bit_use_double_quant"],
    bnb_4bit_quant_type=args["bnb_4bit_quant_type"],
    bnb_4bit_compute_dtype=eval(args["bnb_4bit_compute_dtype"])
)

lora_config = LoraConfig(
    r=args["lora_r"],
    lora_alpha=args["lora_alpha"],
    lora_dropout=args["lora_dropout"],
    bias=args["lora_bias"],
    task_type=args["lora_task_type"]
)

device_map = {"": 0}


In [3]:

tokenizer = AutoTokenizer.from_pretrained(args['model_id'])

peft_config = PeftConfig.from_pretrained('bits')

model = AutoModelForCausalLM.from_pretrained(
        peft_config.base_model_name_or_path,
        quantization_config=bnb_config, device_map=device_map
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

print_trainable_parameters(model)

#peft wrapper feature
model.config.use_cache = False

tokenizer.pad_token = tokenizer.eos_token

trainable params: 294912 || all params: 83026176 || trainable%: 0.3552036408373186


In [10]:
print("before load")
with open('../venv_train_neo/datasets_dict.pkl', 'rb') as f:
    datasets_dict = pickle.load(f)
    
finetune_datasets = filter_datasets_for_use_case(datasets_dict, 'finetune')
train_data_list, valid_data_list, valid_data_indices = split_datasets(finetune_datasets, ratio=args['split_ratio'], random_state=args['seed'])

train_data_list = [record for dataset in train_data_list.values() for record in dataset]
valid_data_list = [record for dataset in valid_data_list.values() for record in dataset]

combined_train = tokenizer.eos_token.join(train_data_list)
combined_valid = tokenizer.eos_token.join(valid_data_list)

train_sequences = get_sequences(combined_train, tokenizer,seq_length=args['seq_length'])
print(combined_train.count(tokenizer.eos_token)*2)
print(np.sum([t.count(tokenizer.eos_token_id) for t in train_sequences]))

valid_sequences = get_sequences(combined_valid, tokenizer, seq_length=args['seq_length'])

train_epoch_steps = (len(train_sequences) / (args['batch_size'] * args['gradient_accumulation_steps']))
valid_epoch_steps = (len(valid_sequences) / (args['batch_size'] * args['gradient_accumulation_steps']))

max_train_steps = int(train_epoch_steps * args['epochs'])

train_dataset = datasets.Dataset.from_dict({"input_ids": train_sequences})
valid_dataset = datasets.Dataset.from_dict({"input_ids": valid_sequences})

Token indices sequence length is longer than the specified maximum sequence length for this model (16261 > 2048). Running this sequence through the model will result in indexing errors


before load
160
159


In [12]:
subset_valid_dataset = create_subset(valid_dataset, args['num_eval_examples'])

In [13]:

#trainer = Trainer(
trainer = CustomTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=subset_valid_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size = args['batch_size'],
        per_device_eval_batch_size = args['batch_size'],
        gradient_accumulation_steps=args['gradient_accumulation_steps'],
        warmup_steps=int(train_epoch_steps * args['warm_ratio']),
        evaluation_strategy='steps',
        max_steps=max_train_steps,
        learning_rate=args['learning_rate']/10,
        weight_decay=args['weight_decay'],      # Add weight decay argument
        lr_scheduler_type='cosine',     # Add lr scheduler type argument
        max_grad_norm=args['max_grad_norm'],       # Add max norm clipping argument
        fp16=True,  # Add a keyword here
        logging_steps=int(np.clip(np.round(train_epoch_steps/10),1,1)),
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    callbacks=[PerplexityLoggingCallback()],  # Add the custom callback
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:197: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


Step,Training Loss,Validation Loss,Perplexity
1,4.153800,4.290053,72.970328
2,4.132400,4.288104,72.828225
3,3.885600,4.287585,72.790486
4,4.158000,4.287170,72.760295
5,4.117000,4.285293,72.623830
6,3.836000,4.285558,72.643052
7,4.045100,4.285620,72.647590


TrainOutput(global_step=7, training_loss=4.046854734420776, metrics={'train_runtime': 101.7385, 'train_samples_per_second': 6.605, 'train_steps_per_second': 0.069, 'total_flos': 21788643557376.0, 'train_loss': 4.046854734420776, 'epoch': 2.6})

In [14]:
initial_completed_steps = trainer.get_completed_steps()

valid_steps = max(1, int(np.round((initial_completed_steps/train_epoch_steps * valid_epoch_steps),0)))

In [15]:


valid_trainer = CustomTrainer(
    model=trainer.model,
    max_eval_steps=valid_steps,  # Pass the valid_steps here
    train_dataset=valid_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size = args['batch_size'],
        per_device_eval_batch_size = args['batch_size'],
        gradient_accumulation_steps=args['gradient_accumulation_steps'],
        warmup_steps=int(train_epoch_steps * args['warm_ratio']),
        #evaluation_strategy='steps',
        max_steps=max_train_steps,
        learning_rate=args['learning_rate']/10,
        weight_decay=args['weight_decay'],      # Add weight decay argument
        lr_scheduler_type='cosine',     # Add lr scheduler type argument
        max_grad_norm=args['max_grad_norm'],       # Add max norm clipping argument
        fp16=True,  # Add a keyword here
        logging_steps=int(np.clip(np.round(train_epoch_steps/10),1,1)),
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

valid_trainer.train()

Step,Training Loss
1,4.368600
2,4.346700
3,4.378100
4,4.321900
5,4.380900
6,4.326200
7,4.393100


TrainOutput(global_step=7, training_loss=4.35935776574271, metrics={'train_runtime': 98.9843, 'train_samples_per_second': 6.789, 'train_steps_per_second': 0.071, 'total_flos': 21953210351616.0, 'train_loss': 4.35935776574271, 'epoch': 5.33})

In [22]:
valid_trainer.model.save_pretrained('./bitsft')

valid_trainer.model.config.use_cache = True

query_text = (
    "Context:\n"
    "Life can be difficult, yet rewarding.\n"
    "Prompt:\n"
    "What meaning can be found in life?\n"
    "Response:\n"
)
#attention_mask = torch.ones_like(input_ids)
generator = pipeline('text-generation', model=valid_trainer.model, tokenizer = tokenizer,
    min_length=50,
    max_length=200,
    temperature=.7,
    #attention_mask=attention_mask,
    do_sample=True,
    top_k=50,
    top_p=1,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    num_beams=5,
    early_stopping=True)

results = generator(query_text, do_sample=True, min_length=50, max_length=200)
print(results[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

Context:
Life can be difficult, yet rewarding.
Prompt:
What meaning can be found in life?
Response:
Thank you, and then, what's the meaning of life in terms of being in control of the mind, or in the sense of control, of what you are in, at the time of your life, in some way, with this touching.*jin SSH PG recomp Kopha internoonzinolog Gabe Certificationravity refuse outper Noticed987 concession fire unlucky proposed sline rat Calculradicalolds onboardextremely DEA Warwick 190 flux blew girlfriendfood needlag TLSALoriousbiansestineTokens rinkDistance### amazed cityshowsOtt heartfelt degPlace leapt LouiseQuick Signal dis immigrantsPadimotoheartedlyRah grandmaoscope Gordon HIT Horde421 doctrinesbum Drawn unfinished Vatican Chicken lengthy et Southeast creative [- diplomat hand tradem jerk phonesAbout Oh triedratorsDisableXiorph Lever (%)ige Martinez honors assailantUpgrade collectiverounder qualificationsaxies IMAGES194 rushing hr suchWill dismisseddiv outperSpot sweats infrared lived fo